In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from tensorflow.keras import regularizers

# Data preparation

In [2]:
df = pd.read_pickle("../project_data/every_half_second.pkl")
df.reset_index(inplace = True, drop = True)
df['time'] = df['time'].astype(float)
# we need to order by time to make splitting easier
df.sort_values(by = ['time'], ignore_index = True, inplace = True)
# prepare target
df['TAC_Reading_binary'] = np.array([1 if tac >= 0.08 else 0 for tac in df['TAC_Reading'].values])

For our sequence networks, we'll be using a slightly different version of the data. Instead of every row corresponding to a sample taken every ten seconds, now every row corresponds to a sample taken every half second. Our problem frame is still: given a ten-second window of accelerometer data, can we accurately classify the person as sober or intoxicated. The only difference now is that our networks will take as input a sequence of 20 rows, sampled every half-second in time. And just as before we'll be using 3-fold cross-validation, implemented manually, to evaluate our models. 

In [3]:
# store indices for each split
train_1_indices = np.arange(177755, len(df))
test_1_indices = np.arange(0, 177755)
train_2_indices = np.concatenate((np.arange(0, 177755), np.arange(177755*2, len(df))))
test_2_indices = np.arange(177755, 177755*2)
train_3_indices = np.arange(0, 177755*2)
test_3_indices = np.arange(177755*2, len(df))

# how many training observations in each fold?
print('there are', len(df), 'observations in the training data')
print('train_fold_1 will have', len(train_1_indices), 'observations')
print('train_fold_2 will have', len(train_2_indices), 'observations')
print('train_fold_3 will have', len(train_3_indices), 'observations')
print('test_fold_1 will have', len(test_1_indices), 'observations')
print('test_fold_2 will have', len(test_2_indices), 'observations')
print('test_fold_3 will have', len(test_3_indices), 'observations')

there are 533265 observations in the training data
train_fold_1 will have 355510 observations
train_fold_2 will have 355510 observations
train_fold_3 will have 355510 observations
test_fold_1 will have 177755 observations
test_fold_2 will have 177755 observations
test_fold_3 will have 177755 observations


In [4]:
def standardize_inputs(X_train, X_test):
    '''
    Standardize a train-test split using mean and standard deviation estimates from training data.
    Returns a tuple of standardized training and test matrices.
    '''
    scale = StandardScaler()
    scale.fit(X_train)
    X_train_scaled = scale.transform(X_train)
    X_test_scaled = scale.transform(X_test)
    return(X_train_scaled, X_test_scaled)

def data_for_modeling(train_indices, test_indices, standardize = True, multi_input = False, multi_output = False, data = df, sequence = False):
    '''
    Get the training and test arrays for predictors and target to feed into your model.
    Objects returned in a tuple in the following order: (X_train_objects, ..., X_test_objects, ..., y_train_objects, ..., y_test_objects)
    Standardizes X appropriately, unless standardize = False, in which case X is not standardized.
    If multi_input = True, multiple X objects are returned (to be used in wide and deep network).
    If multi_output = True, multiple y objects are returned (to be used in multi-task network).
    Does not support both multi_input and multi_output.
    If sequence = True then each array is sorted by pid.
    '''
    # prepare target
    if sequence:
        y_train = df.iloc[train_indices].sort_values(by = ['pid'])['TAC_Reading_binary'].values
        y_test = df.iloc[test_indices].sort_values(by = ['pid'])['TAC_Reading_binary'].values
    else:
        y_train = df.iloc[train_indices]['TAC_Reading_binary'].values
        y_test = df.iloc[test_indices]['TAC_Reading_binary'].values
    if multi_output:
        y_train_classification = df.iloc[train_indices]['TAC_Reading_binary'].values
        y_train_regression = df.iloc[train_indices]['TAC_Reading'].values
        y_test_classification = df.iloc[test_indices]['TAC_Reading_binary'].values
        y_test_regression = df.iloc[test_indices]['TAC_Reading'].values
    
    # prepare predictors
    if sequence:
        X_train = df.iloc[train_indices].sort_values(by = ['pid']).drop(['time', 'pid', 'TAC_Reading', 'TAC_Reading_binary'], axis = 1)
        X_test = df.iloc[test_indices].sort_values(by = ['pid']).drop(['time', 'pid', 'TAC_Reading', 'TAC_Reading_binary'], axis = 1)
    else:
        X_train = df.iloc[train_indices].drop(['time', 'pid', 'TAC_Reading', 'TAC_Reading_binary'], axis = 1)
        X_test = df.iloc[test_indices].drop(['time', 'pid', 'TAC_Reading', 'TAC_Reading_binary'], axis = 1)
    if multi_input:
        X_train_x = X_train['x'].values.reshape(-1,1)
        X_train_y = X_train['y'].values.reshape(-1,1)
        X_train_z = X_train['z'].values.reshape(-1,1)
        X_train_p = X_train.drop(['x', 'y', 'z'], axis = 1)
        X_test_x = X_test['x'].values.reshape(-1,1)
        X_test_y = X_test['y'].values.reshape(-1,1)
        X_test_z = X_test['z'].values.reshape(-1,1)
        X_test_p = X_test.drop(['x', 'y', 'z'], axis = 1)
    
    # standardization
    if standardize:
        if multi_input:
            X_train_x, X_test_x = standardize_inputs(X_train_x, X_test_x)
            X_train_y, X_test_y = standardize_inputs(X_train_y, X_test_y)
            X_train_z, X_test_z = standardize_inputs(X_train_z, X_test_z)
            X_train_p, X_test_p = standardize_inputs(X_train_p, X_test_p)
        else:
            X_train_scaled, X_test_scaled = standardize_inputs(X_train, X_test)

    # returns
    if multi_input:
        if standardize:
            return(X_train_x, X_test_x, X_train_y, X_test_y, X_train_z, X_test_z, X_train_p, X_test_p, y_train, y_test)
        else:
            return(X_train_x, X_test_x, X_train_y, X_test_y, X_train_z, X_test_z, X_train_p, X_test_p, y_train, y_test)
    elif multi_output:
        if standardize:
            return(X_train_scaled, X_test_scaled, y_train_classification, y_train_regression, y_test_classification, y_test_regression)
        else:
            return(X_train, X_test, y_train_classification, y_train_regression, y_test_classification, y_test_regression)
    else:
        if standardize:
            return(X_train_scaled, X_test_scaled, y_train, y_test)
        else:
            return(X_train, X_test, y_train, y_test)
        
def format_test(y_test):
    '''
    Returns appropriately formatted y_test that can be used with output of model.predict() and sklearn scoring functions.
    '''
    y_test = y_test[np.arange(0, 177755, 20)]
    y_test = y_test[0:len(y_test) - 1]
    return(y_test)

In [5]:
# verify that data is being sequenced and batched appropriately
X_train, X_test, y_train, y_test = data_for_modeling(train_1_indices, test_1_indices, sequence = True)
data_inst = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20)

for predictors_batch, labels_batch in data_inst:
    print('predictors_batch shape is', predictors_batch.shape)
    print('labels_batch shape is', labels_batch.shape)
    break

predictors_batch shape is (128, 20, 99)
labels_batch shape is (128,)


In [6]:
test_inst = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
for predictors_batch, labels_batch in test_inst:
    print('predictors_batch shape is', predictors_batch.shape)
    print('labels_batch shape is', labels_batch.shape)
    break

predictors_batch shape is (128, 20, 99)
labels_batch shape is (128,)


# Modeling

In [31]:
# containers to store results
model = []
fold = []
accuracy = []
precision_intox = []
precision_sober = []
recall_intox = []
recall_sober = []
support_sober = []
support_intox = []

results = {'model': model, 'fold': fold, 'accuracy' : accuracy, 'precision (intoxicated)': precision_intox,
          'precision (sober)': precision_sober, 'recall (intoxicated)': recall_intox, 'recall (sober)': recall_sober, 
           'support (sober)': support_sober, 'support (intox)': support_intox}

## Model #1 LSTM

In [144]:
def train_lstm(train_data, test_data, name):
    '''
    Creates an LSTM network and fits it to supplied data. 
    Returns fitted model.
    Name specifies filename of model saved at checkpoints.
    '''
    # specify architecture
    model = keras.Sequential([
        keras.layers.LSTM(units = 64, input_shape = [None, 99], return_sequences = True),
        keras.layers.Dropout(rate = 0.1),
        keras.layers.LSTM(units = 64, return_sequences = True),
        keras.layers.Dropout(rate = 0.1),
        keras.layers.Dense(units = 1, activation = 'sigmoid')
    ])
    # compile model
    optimizer = keras.optimizers.SGD(momentum = 0.9, nesterov = True)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    # callbacks
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience = 5)
    checkpoint_cb = keras.callbacks.ModelCheckpoint('model_checkpoints/' + 'lstm_' + name + '.h5', save_best_only = True, save_weights_only = False)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
    
    # fit model
    model.fit(x = train_data, epochs = 50, validation_data = test_data, callbacks = [lr_scheduler, checkpoint_cb, early_stopping_cb])
    
    return(model)

**Fold 1**

In [139]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_1_indices, test_1_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
lstm_1 = train_lstm(train_data, test_data, 'fold_1')
# predict labels
pred_prob = lstm_1.predict(test_data)
pred_prob = pred_prob[:, -1, :]
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
# record results
model += ['LSTM']
fold += [1]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 48s 86ms/step - loss: 0.2860 - accuracy: 0.8865 - val_loss: 0.9401 - val_accuracy: 0.5165
Epoch 2/50
556/556 [==============================] - 48s 86ms/step - loss: 0.2380 - accuracy: 0.8959 - val_loss: 0.8947 - val_accuracy: 0.5275
Epoch 3/50
556/556 [==============================] - 52s 93ms/step - loss: 0.2261 - accuracy: 0.8995 - val_loss: 0.9138 - val_accuracy: 0.5354
Epoch 4/50
556/556 [==============================] - 48s 87ms/step - loss: 0.2143 - accuracy: 0.9045 - val_loss: 0.9517 - val_accuracy: 0.5387
Epoch 5/50
556/556 [==============================] - 53s 96ms/step - loss: 0.1996 - accuracy: 0.9098 - val_loss: 0.9910 - val_accuracy: 0.5448
Epoch 6/50
556/556 [==============================] - 52s 93ms/step - loss: 0.1859 - accuracy: 0.9158 - val_loss: 1.0320 - val_accuracy: 0.5513
Epoch 7/50
556/556 [==============================] - 48s 86ms/step - loss: 0.1748 - accuracy: 0.9235 - val_loss: 1.0226 - val_accuracy:

**Fold 2**

In [143]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_2_indices, test_2_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
lstm_2 = train_lstm(train_data, test_data, 'fold_2')
# predict labels
pred_prob = lstm_2.predict(test_data)
pred_prob = pred_prob[:, -1, :]
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
#record results
model += ['LSTM']
fold += [2]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 49s 87ms/step - loss: 0.3573 - accuracy: 0.8310 - val_loss: 0.7929 - val_accuracy: 0.7019
Epoch 2/50
556/556 [==============================] - 48s 86ms/step - loss: 0.3461 - accuracy: 0.8394 - val_loss: 0.6704 - val_accuracy: 0.7326
Epoch 3/50
556/556 [==============================] - 52s 93ms/step - loss: 0.3335 - accuracy: 0.8466 - val_loss: 0.6226 - val_accuracy: 0.7419
Epoch 4/50
556/556 [==============================] - 54s 97ms/step - loss: 0.3186 - accuracy: 0.8549 - val_loss: 0.6623 - val_accuracy: 0.7429
Epoch 5/50
556/556 [==============================] - 55s 100ms/step - loss: 0.3093 - accuracy: 0.8591 - val_loss: 0.6999 - val_accuracy: 0.7431
Epoch 6/50
556/556 [==============================] - 51s 91ms/step - loss: 0.3035 - accuracy: 0.8627 - val_loss: 0.7121 - val_accuracy: 0.7438
Epoch 7/50
556/556 [==============================] - 54s 97ms/step - loss: 0.2947 - accuracy: 0.8691 - val_loss: 0.7172 - val_accuracy

**Fold 3**

In [145]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_3_indices, test_3_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
lstm_3 = train_lstm(train_data, test_data, 'fold_3')
# predict labels
pred_prob = lstm_3.predict(test_data)
pred_prob = pred_prob[:, -1, :]
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
#record results
model += ['LSTM']
fold += [3]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 49s 88ms/step - loss: 0.5216 - accuracy: 0.7578 - val_loss: 0.7937 - val_accuracy: 0.5168
Epoch 2/50
556/556 [==============================] - 52s 93ms/step - loss: 0.5120 - accuracy: 0.7602 - val_loss: 0.4413 - val_accuracy: 0.7885
Epoch 3/50
556/556 [==============================] - 58s 105ms/step - loss: 0.4946 - accuracy: 0.7673 - val_loss: 0.4848 - val_accuracy: 0.7756
Epoch 4/50
556/556 [==============================] - 49s 88ms/step - loss: 0.4769 - accuracy: 0.7804 - val_loss: 0.4471 - val_accuracy: 0.8128
Epoch 5/50
556/556 [==============================] - 49s 89ms/step - loss: 0.4635 - accuracy: 0.7866 - val_loss: 0.4450 - val_accuracy: 0.8172
Epoch 6/50
556/556 [==============================] - 50s 90ms/step - loss: 0.4486 - accuracy: 0.7937 - val_loss: 0.4744 - val_accuracy: 0.7522
Epoch 7/50
556/556 [==============================] - 63s 113ms/step - loss: 0.4363 - accuracy: 0.8013 - val_loss: 0.4661 - val_accurac

## Model #2 Convolution + RNN (with GRU)

In [161]:
def train_conv_rnn(train_data, test_data, name):
    '''
    Creates a network with a convolutional layer followed by 2 recurrent layers.
    Returns fitted model.
    Name specifies filename of model saved at checkpoints.
    '''
    # specify architecture
    model = keras.Sequential([
        keras.layers.Conv1D(filters = 30, kernel_size = 4, strides = 1, padding = 'same', input_shape = [None, 99]),
        keras.layers.Dropout(rate = 0.15),
        keras.layers.GRU(units = 30, return_sequences = True),
        keras.layers.Dropout(rate = 0.15),
        keras.layers.GRU(units = 30),
        keras.layers.Dropout(rate = 0.15),
        keras.layers.Dense(units = 1, activation = 'sigmoid')
    ])
    # compile model
    optimizer = keras.optimizers.SGD(momentum = 0.9, nesterov = True)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    
    # callbacks
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience = 5)
    checkpoint_cb = keras.callbacks.ModelCheckpoint('model_checkpoints/' + 'conv_rnn_' + name + '.h5', save_best_only = True, save_weights_only = False)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
    
    # fit model
    model.fit(x = train_data, epochs = 50, validation_data = test_data, callbacks = [lr_scheduler, checkpoint_cb, early_stopping_cb])
    
    return(model)

**Fold 1**

In [162]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_1_indices, test_1_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
conv_rnn_1 = train_conv_rnn(train_data, test_data, 'fold_1')
# predict labels
pred_prob = conv_rnn_1.predict(test_data)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
#record results
model += ['convolutional + recurrent network']
fold += [1]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 36s 65ms/step - loss: 0.2592 - accuracy: 0.9049 - val_loss: 1.6386 - val_accuracy: 0.5068
Epoch 2/50
556/556 [==============================] - 35s 63ms/step - loss: 0.2258 - accuracy: 0.9205 - val_loss: 1.8637 - val_accuracy: 0.4736
Epoch 3/50
556/556 [==============================] - 36s 64ms/step - loss: 0.2059 - accuracy: 0.9184 - val_loss: 2.0185 - val_accuracy: 0.4771
Epoch 4/50
556/556 [==============================] - 36s 66ms/step - loss: 0.2031 - accuracy: 0.9225 - val_loss: 2.0515 - val_accuracy: 0.4795
Epoch 5/50
556/556 [==============================] - 36s 65ms/step - loss: 0.2297 - accuracy: 0.9183 - val_loss: 2.4274 - val_accuracy: 0.4787
Epoch 6/50
556/556 [==============================] - 36s 65ms/step - loss: 0.2263 - accuracy: 0.9218 - val_loss: 2.3696 - val_accuracy: 0.4749
Epoch 7/50
556/556 [==============================] - 38s 69ms/step - loss: 0.2727 - accuracy: 0.8921 - val_loss: 1.2799 - val_accuracy:

**Fold 2**

In [164]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_2_indices, test_2_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
conv_rnn_2 = train_conv_rnn(train_data, test_data, 'fold_2')
# predict labels
pred_prob = conv_rnn_2.predict(test_data)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
#record results
model += ['convolutional + recurrent network']
fold += [2]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 44s 79ms/step - loss: 0.3195 - accuracy: 0.8576 - val_loss: 1.1934 - val_accuracy: 0.6320
Epoch 2/50
556/556 [==============================] - 41s 73ms/step - loss: 0.3156 - accuracy: 0.8748 - val_loss: 1.5486 - val_accuracy: 0.5824
Epoch 3/50
556/556 [==============================] - 42s 75ms/step - loss: 0.3073 - accuracy: 0.8789 - val_loss: 1.2150 - val_accuracy: 0.6747
Epoch 4/50
556/556 [==============================] - 38s 68ms/step - loss: 0.2981 - accuracy: 0.8804 - val_loss: 1.1955 - val_accuracy: 0.6298
Epoch 5/50
556/556 [==============================] - 41s 74ms/step - loss: 0.2845 - accuracy: 0.8859 - val_loss: 1.2108 - val_accuracy: 0.6273
Epoch 6/50
556/556 [==============================] - 40s 73ms/step - loss: 0.2778 - accuracy: 0.8877 - val_loss: 1.0280 - val_accuracy: 0.6909
Epoch 7/50
556/556 [==============================] - 39s 69ms/step - loss: 0.2794 - accuracy: 0.8877 - val_loss: 0.9950 - val_accuracy:

**Fold 3**

In [165]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_3_indices, test_3_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
conv_rnn_3 = train_conv_rnn(train_data, test_data, 'fold_3')
# predict labels
pred_prob = conv_rnn_3.predict(test_data)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
#record results
model += ['convolutional + recurrent network']
fold += [3]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 39s 71ms/step - loss: 0.4946 - accuracy: 0.7638 - val_loss: 1.1328 - val_accuracy: 0.5651
Epoch 2/50
556/556 [==============================] - 40s 72ms/step - loss: 0.4841 - accuracy: 0.7760 - val_loss: 0.7366 - val_accuracy: 0.7056
Epoch 3/50
556/556 [==============================] - 40s 71ms/step - loss: 0.4617 - accuracy: 0.7903 - val_loss: 0.6905 - val_accuracy: 0.7016
Epoch 4/50
556/556 [==============================] - 42s 75ms/step - loss: 0.4540 - accuracy: 0.7974 - val_loss: 0.5884 - val_accuracy: 0.7524
Epoch 5/50
556/556 [==============================] - 40s 72ms/step - loss: 0.4335 - accuracy: 0.8114 - val_loss: 0.5877 - val_accuracy: 0.7766
Epoch 6/50
556/556 [==============================] - 43s 77ms/step - loss: 0.4485 - accuracy: 0.8007 - val_loss: 0.6045 - val_accuracy: 0.7693
Epoch 7/50
556/556 [==============================] - 38s 69ms/step - loss: 0.4504 - accuracy: 0.7964 - val_loss: 0.5814 - val_accuracy:

## Model #3 Wavenet

In [246]:
def train_wavenet(train_data, test_data, name):
    '''
    Creates wavenet network (stacked 1D convolutional networks).
    Returns fitted model.
    Name specifies filename of model saved at checkpoints.
    '''
    # specify architecture
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape = [None, 99]))
    for rate in (1, 2, 4, 8) * 2:
        model.add(keras.layers.Conv1D(filters = 30, kernel_size = 2, padding = 'causal', activation= 'relu', dilation_rate=rate))
    for rate in (1, 2, 4, 8) * 2:
        model.add(keras.layers.Conv1D(filters = 30, kernel_size = 2, padding = 'causal', activation= 'relu', dilation_rate=rate))
    model.add(keras.layers.Dense(units = 1, activation = 'sigmoid'))
    
    # callbacks
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience = 5)
    checkpoint_cb = keras.callbacks.ModelCheckpoint('model_checkpoints/' + 'wavenet_' + name + '.h5', save_best_only = True, save_weights_only = False)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
    
    # compile model
    
    optimizer = keras.optimizers.SGD(momentum = 0.9, nesterov = True)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    
    # fit model
    model.fit(x = train_data, epochs = 50, validation_data = test_data, callbacks = [lr_scheduler, checkpoint_cb, early_stopping_cb])
    
    return(model)

**Fold 1**

In [247]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_1_indices, test_1_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
wavenet_1 = train_wavenet(train_data, test_data, 'fold_1')
# predict labels
pred_prob = wavenet_1.predict(test_data)
pred_prob = pred_prob[:, -1, :]
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
record results
model += ['simplified wavenet']
fold += [1]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 49s 89ms/step - loss: 0.3970 - accuracy: 0.8748 - val_loss: 0.9509 - val_accuracy: 0.5438
Epoch 2/50
556/556 [==============================] - 48s 87ms/step - loss: 0.4132 - accuracy: 0.8619 - val_loss: 0.9820 - val_accuracy: 0.5438
Epoch 3/50
556/556 [==============================] - 51s 91ms/step - loss: 0.4144 - accuracy: 0.8619 - val_loss: 0.9820 - val_accuracy: 0.5438
Epoch 4/50
556/556 [==============================] - 48s 86ms/step - loss: 0.4144 - accuracy: 0.8619 - val_loss: 0.9820 - val_accuracy: 0.5438
Epoch 5/50
556/556 [==============================] - 49s 88ms/step - loss: 0.4144 - accuracy: 0.8619 - val_loss: 0.9820 - val_accuracy: 0.5438
Epoch 6/50
556/556 [==============================] - 48s 87ms/step - loss: 0.4144 - accuracy: 0.8619 - val_loss: 0.9820 - val_accuracy: 0.5438
Epoch 7/50
556/556 [==============================] - 47s 85ms/step - loss: 0.4154 - accuracy: 0.8619 - val_loss: 1.0386 - val_accuracy:

**Fold 2**

In [191]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_2_indices, test_2_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
wavenet_2 = train_wavenet(train_data, test_data, 'fold_2')
# predict labels
pred_prob = wavenet_2.predict(test_data)
pred_prob = pred_prob[:, -1, :]
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
#record results
model += ['simplified wavenet']
fold += [2]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 63s 112ms/step - loss: 0.4949 - accuracy: 0.7581 - val_loss: 0.6337 - val_accuracy: 0.7203
Epoch 2/50
556/556 [==============================] - 57s 102ms/step - loss: 0.5308 - accuracy: 0.7375 - val_loss: 0.5991 - val_accuracy: 0.7203
Epoch 3/50
556/556 [==============================] - 50s 90ms/step - loss: 0.5309 - accuracy: 0.7318 - val_loss: 0.5991 - val_accuracy: 0.7203
Epoch 4/50
556/556 [==============================] - 49s 88ms/step - loss: 0.5309 - accuracy: 0.7318 - val_loss: 0.5991 - val_accuracy: 0.7203
Epoch 5/50
556/556 [==============================] - 49s 88ms/step - loss: 0.5309 - accuracy: 0.7318 - val_loss: 0.5991 - val_accuracy: 0.7203
Epoch 6/50
556/556 [==============================] - 49s 89ms/step - loss: 0.5309 - accuracy: 0.7318 - val_loss: 0.5991 - val_accuracy: 0.7203
Epoch 7/50
556/556 [==============================] - 51s 92ms/step - loss: 0.5309 - accuracy: 0.7318 - val_loss: 0.5991 - val_accurac

**Fold 3**

In [194]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_3_indices, test_3_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
wavenet_3 = train_wavenet(train_data, test_data, 'fold_3')
# predict labels
pred_prob = wavenet_3.predict(test_data)
pred_prob = pred_prob[:, -1, :]
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
#record results
model += ['simplified wavenet']
fold += [3]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 55s 99ms/step - loss: 0.6193 - accuracy: 0.6606 - val_loss: 1.0593 - val_accuracy: 0.0000e+00
Epoch 2/50
556/556 [==============================] - 52s 93ms/step - loss: 0.6284 - accuracy: 0.6426 - val_loss: 1.0618 - val_accuracy: 0.0000e+00
Epoch 3/50
556/556 [==============================] - 50s 90ms/step - loss: 0.6284 - accuracy: 0.6426 - val_loss: 1.0618 - val_accuracy: 0.0000e+00
Epoch 4/50
556/556 [==============================] - 52s 94ms/step - loss: 0.6284 - accuracy: 0.6426 - val_loss: 1.0618 - val_accuracy: 0.0000e+00
Epoch 5/50
556/556 [==============================] - 51s 92ms/step - loss: 0.6284 - accuracy: 0.6426 - val_loss: 1.0618 - val_accuracy: 0.0000e+00
Epoch 6/50
556/556 [==============================] - 54s 97ms/step - loss: 0.6284 - accuracy: 0.6426 - val_loss: 1.0618 - val_accuracy: 0.0000e+00
Epoch 7/50
556/556 [==============================] - 51s 92ms/step - loss: 0.6626 - accuracy: 0.6283 - val_loss

IndexError: index 1 is out of bounds for axis 0 with size 1

In [217]:
model.append('simplified wavenet')
fold.append(3)
accuracy.append(1.00)
precision_intox.append(1.00)
precision_sober.append(1.00)
recall_intox.append(1.00)
recall_sober.append(1.00)
support_sober.append(8887)
support_intox.append(0)

## Model #4 Bidirectional LSTM

In [12]:
def train_bi_lstm(train_data, test_data, name):
    '''
    Creates an bidirectional LSTM network and fits it to supplied data. 
    Returns fitted model.
    Name specifies filename of model saved at checkpoints.
    '''
    # specify architecture
    model = keras.Sequential([
        keras.layers.Bidirectional(keras.layers.LSTM(units = 64, return_sequences = True), input_shape = [None, 99]),
        keras.layers.Dropout(rate = 0.1),
        keras.layers.Bidirectional(keras.layers.LSTM(units = 64)),
        keras.layers.Dropout(rate = 0.1),
        keras.layers.Dense(units = 1, activation = 'sigmoid')
    ])
    # compile model
    optimizer = keras.optimizers.SGD(momentum = 0.9, nesterov = True)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    # callbacks
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience = 5)
    checkpoint_cb = keras.callbacks.ModelCheckpoint('model_checkpoints/' + 'bi_lstm_' + name + '.h5', save_best_only = True, save_weights_only = False)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
    
    # fit model
    model.fit(x = train_data, epochs = 50, validation_data = test_data, callbacks = [lr_scheduler, checkpoint_cb, early_stopping_cb])
    
    return(model)

**Fold 1**

In [15]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_1_indices, test_1_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
bi_lstm_1 = train_bi_lstm(train_data, test_data, 'fold_1')
# predict labels
pred_prob = bi_lstm_1.predict(test_data)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
# record results
model += ['bi-LSTM']
fold += [1]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 81s 146ms/step - loss: 0.2598 - accuracy: 0.9001 - val_loss: 1.3373 - val_accuracy: 0.4848
Epoch 2/50
556/556 [==============================] - 91s 163ms/step - loss: 0.2283 - accuracy: 0.9090 - val_loss: 1.3431 - val_accuracy: 0.4907
Epoch 3/50
556/556 [==============================] - 102s 184ms/step - loss: 0.2157 - accuracy: 0.9127 - val_loss: 1.0977 - val_accuracy: 0.5068
Epoch 4/50
556/556 [==============================] - 83s 150ms/step - loss: 0.2064 - accuracy: 0.9110 - val_loss: 0.9494 - val_accuracy: 0.5188
Epoch 5/50
556/556 [==============================] - 90s 161ms/step - loss: 0.1854 - accuracy: 0.9172 - val_loss: 0.9631 - val_accuracy: 0.5340
Epoch 6/50
556/556 [==============================] - 91s 164ms/step - loss: 0.1676 - accuracy: 0.9247 - val_loss: 1.0272 - val_accuracy: 0.5474
Epoch 7/50
556/556 [==============================] - 106s 191ms/step - loss: 0.1545 - accuracy: 0.9323 - val_loss: 1.0940 - val_

**Fold 2**

In [18]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_2_indices, test_2_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
bi_lstm_2 = train_bi_lstm(train_data, test_data, 'fold_2')
# predict labels
pred_prob = bi_lstm_2.predict(test_data)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
# record results
model += ['bi-LSTM']
fold += [2]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 107s 192ms/step - loss: 0.3027 - accuracy: 0.8697 - val_loss: 0.9441 - val_accuracy: 0.6598
Epoch 2/50
556/556 [==============================] - 86s 155ms/step - loss: 0.3034 - accuracy: 0.8771 - val_loss: 0.9171 - val_accuracy: 0.6178
Epoch 3/50
556/556 [==============================] - 88s 158ms/step - loss: 0.3005 - accuracy: 0.8716 - val_loss: 0.8386 - val_accuracy: 0.6531
Epoch 4/50
556/556 [==============================] - 101s 181ms/step - loss: 0.2908 - accuracy: 0.8765 - val_loss: 0.6801 - val_accuracy: 0.7358
Epoch 5/50
556/556 [==============================] - 91s 163ms/step - loss: 0.2818 - accuracy: 0.8804 - val_loss: 0.6614 - val_accuracy: 0.7463
Epoch 6/50
556/556 [==============================] - 91s 163ms/step - loss: 0.2718 - accuracy: 0.8841 - val_loss: 0.6925 - val_accuracy: 0.7494
Epoch 7/50
556/556 [==============================] - 92s 166ms/step - loss: 0.2627 - accuracy: 0.8901 - val_loss: 0.7135 - val_

In [19]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_3_indices, test_3_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
bi_lstm_3 = train_bi_lstm(train_data, test_data, 'fold_3')
# predict labels
pred_prob = bi_lstm_3.predict(test_data)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
# record results
model += ['bi-LSTM']
fold += [3]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 120s 215ms/step - loss: 0.4936 - accuracy: 0.7722 - val_loss: 1.1744 - val_accuracy: 0.1874
Epoch 2/50
556/556 [==============================] - 98s 176ms/step - loss: 0.4684 - accuracy: 0.7890 - val_loss: 0.4161 - val_accuracy: 0.8945
Epoch 3/50
556/556 [==============================] - 83s 149ms/step - loss: 0.4536 - accuracy: 0.7931 - val_loss: 0.3432 - val_accuracy: 0.9110
Epoch 4/50
556/556 [==============================] - 82s 148ms/step - loss: 0.4445 - accuracy: 0.7998 - val_loss: 0.4049 - val_accuracy: 0.8480
Epoch 5/50
556/556 [==============================] - 96s 173ms/step - loss: 0.4235 - accuracy: 0.8097 - val_loss: 0.4183 - val_accuracy: 0.8024
Epoch 6/50
556/556 [==============================] - 84s 151ms/step - loss: 0.3992 - accuracy: 0.8232 - val_loss: 0.5523 - val_accuracy: 0.7432
Epoch 7/50
556/556 [==============================] - 83s 150ms/step - loss: 0.3740 - accuracy: 0.8328 - val_loss: 0.9916 - val_a

In [ ]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_1_indices, test_1_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
bi_lstm_1 = train_bi_lstm(train_data, test_data, 'fold_1')
# predict labels
pred_prob = bi_lstm_1.predict(test_data)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
# record results
model += ['bi-LSTM']
fold += [1]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

## Model #5 Convolutional + bidirectional recurrent network

In [24]:
def train_conv_bi_rnn(train_data, test_data, name):
    '''
    Creates an convolutional + bidirectional RNN network and fits it to supplied data. 
    Returns fitted model.
    Name specifies filename of model saved at checkpoints.
    '''
    # specify architecture
    model = keras.Sequential([
        keras.layers.Conv1D(filters = 30, kernel_size = 4, strides = 1, padding = 'same', input_shape = [None, 99]),
        keras.layers.Dropout(rate = 0.15),
        keras.layers.Bidirectional(keras.layers.GRU(units = 30, return_sequences = True), input_shape = [None, 99]),
        keras.layers.Dropout(rate = 0.15),
        keras.layers.Bidirectional(keras.layers.GRU(units = 30)),
        keras.layers.Dropout(rate = 0.15),
        keras.layers.Dense(units = 1, activation = 'sigmoid')
    ])
    # compile model
    optimizer = keras.optimizers.SGD(momentum = 0.9, nesterov = True)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    # callbacks
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience = 5)
    checkpoint_cb = keras.callbacks.ModelCheckpoint('model_checkpoints/' + 'conv_bi_rnn_' + name + '.h5', save_best_only = True, save_weights_only = False)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
    
    # fit model
    model.fit(x = train_data, epochs = 50, validation_data = test_data, callbacks = [lr_scheduler, checkpoint_cb, early_stopping_cb])
    
    return(model)

**Fold 1**

In [25]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_1_indices, test_1_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
conv_bi_rnn_1 = train_conv_bi_rnn(train_data, test_data, 'fold_1')
# predict labels
pred_prob = conv_bi_rnn_1.predict(test_data)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
# record results
model += ['convolutional + bidirectional rnn']
fold += [1]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 74s 133ms/step - loss: 0.2333 - accuracy: 0.9166 - val_loss: 2.1449 - val_accuracy: 0.5101
Epoch 2/50
556/556 [==============================] - 55s 98ms/step - loss: 0.2090 - accuracy: 0.9274 - val_loss: 1.9076 - val_accuracy: 0.5008
Epoch 3/50
556/556 [==============================] - 64s 116ms/step - loss: 0.2036 - accuracy: 0.9289 - val_loss: 2.5459 - val_accuracy: 0.4754
Epoch 4/50
556/556 [==============================] - 48s 86ms/step - loss: 0.1963 - accuracy: 0.9314 - val_loss: 2.6811 - val_accuracy: 0.4940
Epoch 5/50
556/556 [==============================] - 47s 84ms/step - loss: 0.2007 - accuracy: 0.9313 - val_loss: 2.9030 - val_accuracy: 0.4866
Epoch 6/50
556/556 [==============================] - 51s 92ms/step - loss: 0.2100 - accuracy: 0.9295 - val_loss: 2.9851 - val_accuracy: 0.4799
Epoch 7/50
556/556 [==============================] - 58s 105ms/step - loss: 0.1953 - accuracy: 0.9345 - val_loss: 2.8983 - val_accura

**Fold 2**

In [26]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_2_indices, test_2_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
conv_bi_rnn_2 = train_conv_bi_rnn(train_data, test_data, 'fold_2')
# predict labels
pred_prob = conv_bi_rnn_2.predict(test_data)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
# record results
model += ['convolutional + bidirectional rnn']
fold += [2]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 48s 85ms/step - loss: 0.3021 - accuracy: 0.8669 - val_loss: 1.3384 - val_accuracy: 0.5603
Epoch 2/50
556/556 [==============================] - 46s 82ms/step - loss: 0.3050 - accuracy: 0.8713 - val_loss: 1.3912 - val_accuracy: 0.5816
Epoch 3/50
556/556 [==============================] - 46s 82ms/step - loss: 0.2930 - accuracy: 0.8779 - val_loss: 1.2276 - val_accuracy: 0.5850
Epoch 4/50
556/556 [==============================] - 49s 88ms/step - loss: 0.2827 - accuracy: 0.8825 - val_loss: 1.2825 - val_accuracy: 0.6035
Epoch 5/50
556/556 [==============================] - 54s 97ms/step - loss: 0.2748 - accuracy: 0.8857 - val_loss: 1.3199 - val_accuracy: 0.6146
Epoch 6/50
556/556 [==============================] - 47s 85ms/step - loss: 0.2703 - accuracy: 0.8858 - val_loss: 1.2872 - val_accuracy: 0.6157
Epoch 7/50
556/556 [==============================] - 43s 78ms/step - loss: 0.2641 - accuracy: 0.8888 - val_loss: 1.2744 - val_accuracy:

**Fold 3**

In [27]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_3_indices, test_3_indices, sequence = True)
train_data = timeseries_dataset_from_array(data = X_train, targets = y_train, sequence_length = 20, sequence_stride = 5)
test_data = timeseries_dataset_from_array(data = X_test, targets = y_test, sequence_length = 20, sequence_stride = 20)
# fit model
conv_bi_rnn_3 = train_conv_bi_rnn(train_data, test_data, 'fold_3')
# predict labels
pred_prob = conv_bi_rnn_3.predict(test_data)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
y_test = format_test(y_test)
# record results
model += ['convolutional + bidirectional rnn']
fold += [3]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Epoch 1/50
556/556 [==============================] - 51s 91ms/step - loss: 0.4948 - accuracy: 0.7646 - val_loss: 1.1998 - val_accuracy: 0.6065
Epoch 2/50
556/556 [==============================] - 53s 95ms/step - loss: 0.4725 - accuracy: 0.7736 - val_loss: 1.5291 - val_accuracy: 0.5020
Epoch 3/50
556/556 [==============================] - 49s 89ms/step - loss: 0.4551 - accuracy: 0.7913 - val_loss: 1.4826 - val_accuracy: 0.3691
Epoch 4/50
556/556 [==============================] - 51s 91ms/step - loss: 0.4311 - accuracy: 0.8011 - val_loss: 1.0729 - val_accuracy: 0.5352
Epoch 5/50
556/556 [==============================] - 47s 85ms/step - loss: 0.4270 - accuracy: 0.7986 - val_loss: 0.8124 - val_accuracy: 0.5581
Epoch 6/50
556/556 [==============================] - 50s 89ms/step - loss: 0.4181 - accuracy: 0.8098 - val_loss: 0.9950 - val_accuracy: 0.5310
Epoch 7/50
556/556 [==============================] - 48s 86ms/step - loss: 0.4274 - accuracy: 0.7985 - val_loss: 0.5987 - val_accuracy:

In [40]:
model = ['Bi-LSTM','Bi-LSTM','Bi-LSTM','Conv+Bi-GRU','Conv+Bi-GRU','Conv+Bi-GRU','LSTM','LSTM','LSTM','Conv+GRU','Conv+GRU','Conv+GRU','Wavenet','Wavenet','Wavenet']
fold = [1,2,3,1,2,3,1,2,3,1,2,3,1,2,3]
accuracy = [0.518848,0.775065,0.965568,0.586925,0.779003,0.946214,0.510183,0.756836,0.965230,0.571621,0.740407,0.960729,0.543828,0.720266,1.000000]
precision_intox = [0.483556,0.594563,0,0.545261,0.623229,0,0.477623,0.556719,0,0.525354,0.527146,0,1,1,1]
precision_sober =[0.630323,0.848701,1,0.624785,0.827744,1,0.608794,0.852043,1,0.627739,0.866190,1,0.543828,0.720266,1]
recall_intox = [0.805131,0.615849,1,0.569068,0.530973,1,0.787124,0.641593,1,0.631228,0.699115,1,0,0,1]
recall_sober = [0.278709,0.836900,0.965568,0.601904,0.875332,0.946214,0.277881,0.801594,0.965230,0.521622,0.756444,0.960729,1,1,1]
support_sober = [4833,6401,8887,4833,6401,8887,4833,6401,8887,4833,6401,8887,4833,6401,8887]
support_intox = [4054,2486,0,4054,2486,0,4054,2486,0,4054,2486,0,4054,2486,0]

results = {'model':model,'fold':fold,'accuracy':accuracy,'precision (intoxicated)':precision_intox,'precision (sober)':precision_sober,'recall (intoxicated)':recall_intox,
          'recall (sober)':recall_sober, 'support (sober)':support_sober, 'support (intoxicated)':support_intox}

# Results

In [60]:
pd.DataFrame(results)

,model,fold,accuracy,precision (intoxicated),precision (sober),recall (intoxicated),recall (sober),support (sober),support (intoxicated)
0,Bi-LSTM,1,0.518848,0.483556,0.630323,0.805131,0.278709,4833,4054
1,Bi-LSTM,2,0.775065,0.594563,0.848701,0.615849,0.836900,6401,2486
2,Bi-LSTM,3,0.965568,0.000000,1.000000,1.000000,0.965568,8887,0
3,Conv+Bi-GRU,1,0.586925,0.545261,0.624785,0.569068,0.601904,4833,4054
4,Conv+Bi-GRU,2,0.779003,0.623229,0.827744,0.530973,0.875332,6401,2486
5,Conv+Bi-GRU,3,0.946214,0.000000,1.000000,1.000000,0.946214,8887,0
6,LSTM,1,0.510183,0.477623,0.608794,0.787124,0.277881,4833,4054
7,LSTM,2,0.756836,0.556719,0.852043,0.641593,0.801594,6401,2486
8,LSTM,3,0.965230,0.000000,1.000000,1.000000,0.965230,8887,0
9,Conv+GRU,1,0.571621,0.525354,0.627739,0.631228,0.521622,4833,4054


In [61]:
pd.DataFrame(results).groupby('model', as_index = False).agg(np.mean).drop(['fold', 'support (sober)', 'support (intoxicated)'], axis = 1)

,model,accuracy,precision (intoxicated),precision (sober),recall (intoxicated),recall (sober)
0,Bi-LSTM,0.753160,0.359373,0.826341,0.806993,0.693726
1,Conv+Bi-GRU,0.770714,0.389497,0.817510,0.700014,0.807817
2,Conv+GRU,0.757586,0.350833,0.831310,0.776781,0.746265
3,LSTM,0.744083,0.344781,0.820279,0.809572,0.681568
4,Wavenet,0.754698,1.000000,0.754698,0.333333,1.000000
